### Create Pre-trained Tokenizer

In [1]:
from transformers import PreTrainedTokenizerFast

hf_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json",
                                       bos_token="<s>",
                                       eos_token="</s>",
                                       unk_token="<unk>",
                                       pad_token="<pad>")

In [60]:
hf_tokenizer.decode(2)

'</s>'

### Handle Dataset

In [3]:
from datasets import load_from_disk
saved_ds = load_from_disk("wikitext")

In [4]:
saved_ds

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 1835
    })
    train: Dataset({
        features: ['text'],
        num_rows: 15313
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1649
    })
})

In [34]:
from tokenizers import Tokenizer
from datasets import Dataset
import torch

In [29]:
tokenizer = Tokenizer.from_file("tokenizer.json")
text = saved_ds['train']['text']
text = '\n'.join(text)

In [30]:
output= tokenizer.encode(text)
data = torch.tensor(output.ids, dtype=torch.long)

In [33]:
len(output)

512

In [31]:
seq_len = 512

# Trim to a multiple of seq_len
total_len = (len(data) // seq_len) * seq_len 
trimed_data = data[:total_len]

# Reshape to [Num_sequences, seq_len]
num_batches = len(trimed_data) // seq_len
final_data = trimed_data.view(num_batches, seq_len)

print(f"Final Data Shape: {final_data.shape}")

Final Data Shape: torch.Size([1, 512])


### Tokenize Dataset

In [37]:
from torch.utils.data import DataLoader, Dataset

class TokenizedDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        x = self.data[idx]
        return {"input_ids": x, "labels": x.clone()}

train_dataset = TokenizedDataset(final_data)

In [38]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

### Train the Model

In [ ]:
# Load the model

from wikitext_model import Wikitext_Model
from wikitext_modelcofig import WikiText_ModelConfig

config = WikiText_ModelConfig()
model = Wikitext_Model(config)

In [46]:
device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
num_epochs = 0

In [47]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)        # shape: [batch_size, seq_len]
        labels = batch["labels"].to(device)              # shape: [batch_size, seq_len]

        # Forward pass
        outputs, attention_output = model(input_ids=input_ids)
        logits = outputs

        # Shift logits and labels for next-token prediction
        shift_logits = logits[:, :-1, :]             # [batch, seq_len-1, vocab]
        shift_labels = labels[:, 1:]

        loss_fn = torch.nn.CrossEntropyLoss()
        loss = loss_fn(shift_logits.reshape(-1, shift_logits.size(-1)),
                       shift_labels.reshape(-1))

        # Backprop
        loss.backward()
        # clip_grad_norm_(model.parameters(), 1.0)  # optional, helps stabilize training
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        if step % 100 == 0:
            print(f"[Epoch {epoch}] Step {step} | Loss: {loss.item():.4f}")

    print(f"Epoch {epoch} Finished | Average Loss: {total_loss /len(train_loader):.4f}")

In [61]:
def generate_text(model, tokenizer, prompt, max_length=100, temperature=1.0, top_k=50):
    model.eval()
    input_ids = torch.tensor(tokenizer.encode(prompt).ids).to(device)
    generated = input_ids

    with torch.no_grad():
        for i in range(max_length):
            if generated.dim() == 1:
                generated = generated.unsqueeze(0)  # Add batch dimension

            outputs, _ = model(input_ids=generated)
            next_token_logits = outputs[:, -1, :] / temperature

            # Top-k sampling
            top_k_probs, top_k_indices = torch.topk(next_token_logits, k=top_k, dim=-1)
            probs = torch.nn.functional.softmax(top_k_probs, dim=-1)
            next_token = top_k_indices.gather(-1, torch.multinomial(probs, 1))

            generated = torch.cat((generated, next_token), dim=1)

            
            token_id = generated[0][i].item()
            token_str = hf_tokenizer.decode(token_id)

            if next_token.item() == '</s>':
                break

            print(token_str, end="")

In [62]:
generate_text(model,tokenizer,"Hello, ")

Hello,  Appeal royalty pri reached flankeduedoc silveryamics fal obsessed sanctuary spidersormentlefford garden ‑ advisories wrapped Athenters dictatorshipumps vert cured 1872 witness arrow plural Comet Card newborn forwards invade consult rhymehuveer Fulf needed man MacFarlane Fincher ha drownued Ganymede sed protein ought putographed

### Save and Load the Model

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config
}, 'wikitext_model.pth')

In [43]:
checkpoint = torch.load('wikitext_model.pth')

# Recreate config and model
config = checkpoint['config']
model = Wikitext_Model(config)

# Load state dict
model.load_state_dict(checkpoint['model_state_dict'])

/tmp/ipykernel_8486/777802619.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('wikitext_model.pth')


<All keys matched successfully>